In [ ]:
# Importation et chargement des bibliothèques
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing, cluster

In [ ]:
# Chargement du dataset
retail_df = pd.read_csv("../data/online_retail_II.csv")
retail_df.info()

In [ ]:
retail_df.head()

In [ ]:
# Conversion de la colonne InnvoiceDate en "datetime"
retail_df["InvoiceDateTime"] = pd.to_datetime(retail_df["InvoiceDate"], errors="coerce")
retail_df.info()
retail_df.head()

In [ ]:
# Cleaning du dataframe
# Suppression des clients non identfiés
retail_df.dropna(subset=["Customer ID"], inplace=True)

# Suppression des doublons
retail_df.drop_duplicates(inplace=True)

# Filtrage du dataset pour écarter les valeurs négatives (les retours de produits) sur les colonnes "Quantity" et "Price"
df_filtred = retail_df[(retail_df["Quantity"] > 0) & (retail_df["Price"] > 0)]

# Affichage des modifications
print(f"Les nouvelles dimensions du dataframe filtré : {df_filtred.shape}")
df_filtred.info()
df_filtred.head()

In [ ]:
# Création de la colonne "TotalAmount"
df_filtred["TotalAmount"] = df_filtred["Quantity"] * df_filtred["Price"]

# Définition de la date référence de calcul
ref_date = df_filtred["InvoiceDateTime"].max() + pd.Timedelta(days=1)
print(f"Date de référence pour l'analyse : {ref_date}")

# Détermination du RFM
rfm = df_filtred.groupby("Customer ID").agg({
    "InvoiceDateTime": lambda x : (ref_date - x.max()).days,
    "Invoice":"nunique",
    "TotalAmount":"sum"
})

# Renommage des colonnes pour plus de clarté
rfm.rename(columns={
    "InvoiceDateTime":"Recency",    # RECENCY : Date Ref - Date Max du client
    "Invoice":"Frequency",          # FREQUENCY : Compte les factures UNIQUES
    "TotalAmount":"Monetary"},      # MONETARY : Somme des montants
    inplace=True)

# Affichage
rfm.head()

In [ ]:
# Standardisation du dataset
# Instanciation du Scaler
scaler = preprocessing.StandardScaler()

# Transformation des données
rfm_scaled_array = scaler.fit_transform(rfm)

# Transformation des données standardisées en dataframes
rfm_scaled = pd.DataFrame(data=rfm_scaled_array, index=rfm.index, columns=rfm.columns)

# Affichage
rfm_scaled.head()

In [ ]:
# Clustering avec KMeans : Recherche du nombre de cluster idéal avec la methode elbow
# Définition du nombre de cluster
range_min = 2
range_max = 11
k_range=range(range_min, range_max)

inertia = []

# Recherche du nombre de cluster idéal
for k in k_range :
    kmeans = cluster.KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled)
    inertia.append(kmeans.inertia_)

# Afichage du graphe
plt.figure(figsize=(10,6))
plt.plot(k_range, inertia, marker='o')
plt.title('Méthode du Coude (Elbow Method)')
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('Inertie')
plt.show()

In [ ]:
# Clustering avec KMeans : Choix de 5 comme nombre de cluster idéal
kmeans_final = cluster.KMeans(n_clusters=5, random_state=42)

# Entrainement du modèle avec le nombre de cluster idéal
kmeans_final.fit(rfm_scaled)

# Ajout des labels des clusters au dataset de base (rfm)
rfm["Cluster"] = kmeans_final.labels_

# Affichage
print(rfm["Cluster"].value_counts())
rfm.head()


In [ ]:
# Reformatage du dataframe
rfm.reset_index(inplace=True)

# Identification des clusters
cluster_summary = rfm.groupby("Cluster").agg({
    "Recency": "mean",
    "Frequency": "mean",
    "Monetary": "mean",
    "Customer ID": "count" 
}).round(2) 

# Renommage des colonnes
cluster_summary.rename(columns={"Customer ID":"Count"}, inplace=True)

# Trie et Affichage
print(cluster_summary.sort_values("Monetary", ascending=False))

In [ ]:
# Profilages des clusters
# Dictionnaire de correspondance
segment_names = {
    3: "Champions 🏆",
    2: "Loyal Customers 🥈",
    1: "Prometteurs 🌱",
    4: "À Risque ⚠️",
    0: "Perdus 💤"
}

# On applique les noms dans une nouvelle colonne
rfm['Segment'] = rfm['Cluster'].map(segment_names)

# Vérification finale
print(rfm[['Customer ID', 'Segment', 'Recency', 'Frequency', 'Monetary']].head())

In [ ]:
# Sauvegarde du résultat final dans le dossier data
rfm.to_csv('../data/rfm_segmented.csv', index=False)